In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pysyncon import Dataprep, Synth, AugSynth
import itertools
import numpy as np
import timeit

In [6]:
# All states:
states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
# States belonging to RGGI. Exclude VA here; it was only in RGGI for a short time
# Connecticut, Delaware, Maine, Maryland, Massachusetts, New Hampshire, New Jersey, New York, Pennsylvania, Rhode Island, and Vermont 
rggi_states = ["CT", "DE", "ME", "MD", "MA", "NH", "NJ", "NY", "PA", "RI", "VT"]
# States with cap-and-trade programs as well as AK and HI, which are of course outside the continental US.
other_states = ["CA", "AK", "HI"]
# States not belonging to RGGI or other cap-and-trade programs.
# WA will be included here because its cap-and-trade program was not around until after 2020.
control_states = list(set(states) - set(rggi_states) - set(other_states))

# Verify all fifty states accounted for:
assert(len(rggi_states) + len(control_states) + len(other_states) == 50)

In [7]:
# Read in the dataframe
df = pd.read_csv(os.path.join("..", "..", "..", "SharedData", "total_state_data.csv"))
df.date = pd.to_datetime(df.date)
df = df[(df.date.dt.year>=1990)&(df.date.dt.year<2020)]

In [8]:
# Do some per capita calculations:
df['co2_per_capita']   = df['monthly_emissions']   / df['monthly_population']
df['gdp_per_capita']   = df['gdp_rel_2017_interp'] / df['monthly_population']
df['eprod_per_capita'] = df['monthly_energy_prod'] / df['monthly_population']
df['eflow_per_capita'] = df['monthly_energy_flow'] / df['monthly_population']

Create a new target: 12 month rolling average of per capita CO2 emissions:

In [9]:
df['co2_emissions_per_capita_sma'] = (df['monthly_emissions'] / df['monthly_population']).rolling(window=12).mean()

In [16]:
monthly_features_of_interest = ['monthly_emissions', 'prcp', 'snow', 'tavg', 
        'gdp_rel_2017_interp', 'monthly_energy_prod',
       'monthly_energy_use', 
       'monthly_energy_flow', 
        'monthly_renew_pct', 'monthly_fossil_pct',
       'monthly_pop_density', 'monthly_emissions_sma']

# set number of months to skip at a time when training monthly model.
# month_jumps = 1 means consider all monthly data; month_jumps = 12 means consider one month per year
month_jumps = 1
ma_window = 1
# Set treatment date

treatment_date = '2009-01-01'
preintervention_ma_start_date = str((pd.to_datetime('1990-06-01')+ pd.DateOffset(months=ma_window-1)).strftime('%Y-%m-%d'))

preintervention_ma_range = df.date[(df.date >= preintervention_ma_start_date) & (df.date<treatment_date)][::month_jumps]

def hyperFeatureSearch(numFeatures, filename):
    scores_df = pd.DataFrame(columns=["Features", "AvgLossSyn"])
    scores_df = scores_df.astype('object')

    # Choose features to test
    for features in itertools.combinations(monthly_features_of_interest, numFeatures):
    
        # Loop over RGGI states
        loss_array = np.zeros(len(rggi_states))
        counter = 0
        for rggi_state in rggi_states:
            state_id = rggi_state 
            control_ids = list(set(control_states) - set([state_id]))
            rggi_ids = list(set(rggi_states) - set([state_id]))

            # Stop the notebook if something goes wrong
            assert(state_id not in other_states)
            assert(len(control_ids) + len(rggi_ids) + 1 == 50 - len(other_states))

            # Do computations monthly
            month_jumps = 1

            # Set up ranges
            UL = 2009
            LL_TIME = 1997      # Time range over which to perform fit

            preintervention_time_range = df.date[(df.date.dt.year>=LL_TIME)&(df.date.dt.year<UL)&(df.state==state_id)][::month_jumps]

            years = pd.date_range(start='1997-06-01', end='2019-12-01', freq='MS').strftime('%Y-%m-%d').tolist()[::month_jumps]
            
            special_predictors = [(feature, preintervention_time_range, 'mean') for feature in features]
            
            dataprep_control = Dataprep(
                foo=df,
                predictors=[],
                predictors_op="mean",
                time_predictors_prior=preintervention_ma_range,
                special_predictors=special_predictors,
                dependent="co2_emissions_per_capita_sma",
                unit_variable="state",
                time_variable="date",
                treatment_identifier=state_id,
                controls_identifier= control_ids,
                time_optimize_ssr=preintervention_ma_range
            )
            
            # Do a synthetic control fit to the data using control states
            synth = Synth()

            synth.fit(dataprep=dataprep_control)
            # print(loss_array)
            loss_array[counter] = synth.loss_V
            counter += 1
            
        feature_avg_loss = np.average(loss_array)
        result = pd.DataFrame(data=[[list(features),feature_avg_loss]], columns=["Features", "AvgLossSyn"], dtype='object')
        # result = result.astype('object')
        # result.loc["Features"] = list(features)
        # result.loc["AvgLossSyn"] = feature_avg_loss
        scores_df = pd.concat([scores_df, result])
        # print(scores_df)
        
    output = filename + "_" + str(numFeatures) + ".csv"
    scores_df.to_csv(output)

In [17]:
# hyperFeatureSearch(1, "hyperSearch_scores")
# print("One done")
hyperFeatureSearch(2, "hyperSearch_scores")
print("Two done")
hyperFeatureSearch(3, "hyperSearch_scores")
print("Three done")
hyperFeatureSearch(4, "hyperSearch_scores")
print("Four done")
hyperFeatureSearch(5, "hyperSearch_scores")
print("Five done")

KeyboardInterrupt: 